In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!ls "/content/drive/My Drive/resources"

In [ ]:
%reload_ext autoreload
%autoreload 2
%autosave 120
%matplotlib inline
import os
import string
import re
import pickle
import json

root = '/content/drive/My Drive/Google Project'
resources_dir = root + "/resources"
pkl_files_dir = root + "/pkl_files"
files_directories_dir = root + "/files_directories.json"


Autosaving every 120 seconds


In [ ]:
allowed_chars = list(string.ascii_lowercase) + list(string.ascii_uppercase)
allowed_chars += [ str(d) for d in range( 10 ) ] + [' ']

def get_alphanumeric( line: str ):
  result = ""
  for ch in line:
    if ch in allowed_chars:
      result += ch
  result = re.sub(' +', ' ', result)
  return result.casefold( )

In [ ]:
def pickle_tree(file_name:str, tree: dict) -> None:
  file_path = pkl_files_dir + "/" + file_name
  outfile = open(file_path,'wb')
  pickle.dump(tree, outfile)
  outfile.close()

def unpickle_tree(file_name:str) -> dict:
  file_path = pkl_files_dir + "/" + file_name
  infile = open(file_path,'rb')
  tree = pickle.load(infile)
  infile.close()
  return tree


In [ ]:
def add( root: dict, prefix: str, file_id: str ) -> None:
    current_node = root
    i = 0
    prefix_length = len(prefix)
    if prefix_length <= 0:
        return

    if not prefix.endswith( "\n" ):
        prefix += "\n"
        prefix_length += 1

    ch = prefix[i]
    while ch in current_node and i < prefix_length - 1:
        current_node = current_node[ ch ]
        i += 1
        ch = prefix[i]

    if i == prefix_length - 1:
        if "\n" not in current_node:
            current_node["\n"] = [ ]
        if file_id not in current_node["\n"]:
          current_node[ "\n" ].append( file_id )
        return

    for ch in prefix[ i: prefix_length - 1 ]:
        current_node[ ch ] = { }
        current_node = current_node[ ch ]
    
    if "\n" not in current_node:
          current_node["\n"] = [ ]
    if file_id not in current_node["\n"]:
          current_node[ "\n" ].append( file_id )


def get_end_sentence(cur_node, max_number, letter, sentence = "", suitable_sentences = set() ):
    if letter == '\n' and len( suitable_sentences ) < max_number :
        suitable_sentences.add( ( sentence, cur_node[0] ) )
        return suitable_sentences

    for letter in cur_node:
        if letter != '\n':
            temp = sentence + letter
        get_end_sentence( cur_node[letter], max_number, letter, temp, suitable_sentences)

        if( len( suitable_sentences ) >= max_number):
            break

    return suitable_sentences


def search_complete_sentence( tree_index: dict, prefix: str ):
    if tree_index is None:
        return [ ]
    cur_node = tree_index
    sentence_results = set()
    sentence_result = ""
    for ch in prefix:
        if ch in cur_node:
            sentence_result += ch
            cur_node = cur_node[ ch ]
        else:
            return []

    if '\n' in cur_node:
        file_id = cur_node['\n'][0]
        sentence_results.add( ( sentence_result, file_id ) )

    if ('\n' in cur_node and len(cur_node) > 1) or ('\n' not in cur_node):
        for letter in cur_node:
            if letter == '\n':
                continue
            results = get_end_sentence( cur_node[letter], 5 - len(sentence_results), letter, sentence_result )
            for res in results:
                sentence_results.add( res )
            if len( results ) >= 5:
                break

    return list(results)[:5]

In [ ]:
def load_file_data(file_name: str):
    result = set( )
    directory_path = resources_dir + "/" + file_name
    file_ = open(directory_path, "r")
    lines = file_.readlines()
    for line in lines:
      yield line

def load_files_pathes( file_path: str ):
  if os.path.isfile( file_path ):
        with open( file_path, 'r', encoding = "utf-8" ) as fp:
            return json.load( fp )["Pathes"]
  return []

def load_all_files_in_range( start: int, end: int):
  files_pathes = load_files_pathes( files_directories_dir )[ start: end ]
  allowed_chars = list(string.ascii_lowercase) + [ str(d) for d in range( 10 ) ]
  for ch in allowed_chars:
    root = { }
    for i, file_path in enumerate(files_pathes):
      print( f" ------------------------------------------- i = { i } -------------------------------------" )
      for line in load_file_data( file_path ):
        if line != None and line[0] == ch:
          add( root, get_alphanumeric(line), str(i) )
      pkl_name = ch
      if pkl_name == 'i':
        pkl_name = "i0"
      pickle_tree( f"{pkl_name}.pkl", root )
